# Data loading and exploratory analysis

## Data loading and import modules

In [27]:
import pandas as pd
import numpy as np
import pandas_profiling

from data_utils.load import beautiful_head, get_data, get_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import KFold

In [28]:
train, test = get_data()
#test = pd.read_csv('../data/test.csv', sep=';')

In [42]:
#pandas_profiling.ProfileReport(train)
print len(test.columns)
print len(train.columns)



33
34


In [30]:
train.head()

,id,annee_naissance,annee_permis,marque,puis_fiscale,anc_veh,codepostal,energie_veh,kmage_annuel,crm,...,var14,var15,var16,var17,var18,var19,var20,var21,var22,prime_tot_ttc
299465,299466,1985.0,2005.0,NISSAN,9,0,97106,gazole,11403,63,...,N,0,4,1,22,2,0,0,0,346.85
81922,81923,1988.0,2007.0,FIAT,6,27,28057,gazole,6811,50,...,E,2,3,1,38,1,1,1,0,292.70
143673,143674,1984.0,2003.0,CITROEN,6,1,45063,gazole,7113,95,...,F,1,1,1,31,0,0,1,2,550.75
208449,208450,1977.0,1997.0,PEUGEOT,7,2,70550,gazole,2949,85,...,D,0,1,1,24,0,1,1,3,333.55
293179,293180,1980.0,1999.0,VOLKSWAGEN,5,0,95199,gazole,6767,100,...,G,1,1,2,23,0,1,1,3,643.08


In [31]:
# check null value existance
print train.isnull().sum().any()
print test.isnull().sum().any()

True
True


In [32]:
# replace NaN values with zero
train.fillna(value=0, inplace=True)
test.fillna(value=0, inplace=True)

In [33]:
# check one more time null value existance
print train.isnull().sum().any()
print test.isnull().sum().any()

False
False


## Labels encoding

In [34]:
def onehotencoding(data, features='all'):
    #функция для перевода всех категориальных данных в числа с помощью кодировки
    #import necessary modules
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    import numpy as np
    import pandas as pd
    
    #write only categorical names to variable cat_columns
    #записать названия столбцов, в которых хранятся категориальные переменные
    #из всех фич выбрать только категориальные
    # <YOUR CODE>
    cat_columns = list(data[features].select_dtypes(include=['category','object']))
    
    
    # encode all features using LabelEncoder
    encoded = []
    
    for i in cat_columns:
        enc = LabelEncoder()
        enc.fit(data[i])
        encoded.append(enc.transform(data[i]))


    #run OHE

    for i in range(len(cat_columns)):
        enc = OneHotEncoder(sparse=False)
        enc.fit(encoded[i].reshape(-1, 1))
        encoded[i] = enc.transform(encoded[i].reshape(-1, 1))


        
    # create a list of columns to help create a DF from np array 
    # <YOUR CODE>
    
    
    
    
    
    # create new dataframe
    # <YOUR CODE>
    new_data = pd.get_dummies(data[cat_columns],columns=data[cat_columns].columns,sparse=True)


    
    
    return new_data

In [54]:
# prepare data for label encoding

# first save target in a separate variable and drop index, id, codepostal
# <YOUR CODE>
target_train = train['prime_tot_ttc']
# stack train and test sets for label encoding
# <YOUR CODE>

#find categorical variables for encoding
# <YOUR CODE>
cat_columns = list(train.select_dtypes(include=['category','object']))

In [55]:
# apply ohe function
new_train = onehotencoding(train, cat_columns)
result_train = pd.concat([train, new_train], axis=1)

In [61]:
# devide back new data frame on train and test sets
new_train

,marque_ACMA,marque_ALFA ROMEO,marque_ALPINE RENAULT,marque_AMC,marque_AMPHICAR,marque_ARO,marque_ASIA,marque_ASTON MARTIN,marque_AUDI,marque_AUSTIN,...,var14_F,var14_G,var14_N,var16_0,var16_1,var16_2,var16_3,var16_4,var16_5,var16_NR
299465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
81922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
143673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
208449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
293179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
105712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
150757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
130391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
191619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# you should get train set shape is (300000, 266) and test set shape is (30000, 266)

## Modeling and CV

In [ ]:
#initializing kfold for our cross validation

In [ ]:
# metric mape

In [ ]:
# apply cv and train RandomForestRegressor model

## Make predictions

In [ ]:
#run randomforest on the whole train set
# <YOUR CODE>

#make predictions
# <YOUR CODE>

In [ ]:
# save predictions